In [22]:
# Öncelikle gerekli kütüphaneleri yükleyelim ve işe başlayalım
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Şimdi Kaggle'dan veri setini çekelim. Burada, Kaggle API'sini kullanarak veri setini çekmek için Kaggle API token'ınızın olması gerekiyor.
!pip install kaggle

  [Kaggle Dataset](https://www.kaggle.com/datasets/aikenkazin/ddos-sdn-dataset)

In [6]:
# Ardından, Kaggle'dan veri setini çekebilir ve projenize ekleyebilirsiniz
# Kaggle API'ye erişim için token dosyası yükleyin
from google.colab import files
files.upload()  # Kaggle.json dosyanızı yükleyin

# Kaggle API'yi kullanarak veri setini indirin
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d your_dataset_name
!unzip your_dataset_name.zip

Saving dataset_sdn.csv to dataset_sdn.csv
mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open your_dataset_name.zip, your_dataset_name.zip.zip or your_dataset_name.zip.ZIP.


In [15]:
# Veri setini yükleme
data = pd.read_csv("/content/dataset_sdn.csv")

data.head()

,dt,switch,src,dst,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,...,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,11425,1,10.0.0.1,10.0.0.8,45304,48294064,100,716000000,1.010000e+11,3,...,451,0,UDP,3,143928631,3917,0,0.0,0.0,0
1,11605,1,10.0.0.1,10.0.0.8,126395,134737070,280,734000000,2.810000e+11,2,...,451,0,UDP,4,3842,3520,0,0.0,0.0,0
2,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,1,3795,1242,0,0.0,0.0,0
3,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,2,3688,1492,0,0.0,0.0,0
4,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,3,3413,3665,0,0.0,0.0,0


In [18]:
# Veri setini önişleme
# (Örneğin: eksik değerlerin doldurulması, kategorik değişkenlerin kodlanması, vb.)

# Öznitelikleri ve hedef değişkeni ayırma
X = data.drop("label", axis=1)
y = data["label"]

# Eğitim ve test setlerini oluşturma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
!pip install pyswarms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.6 MB/s eta 0:00:00


In [26]:
import numpy as np
from pyswarms.utils.functions import single_obj as fx

In [34]:
def objective_function(selected_features):
    model = RandomForestClassifier()
    model.fit(X_train.iloc[:, selected_features], y_train)
    y_pred = model.predict(X_test.iloc[:, selected_features])
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Negatif doğruluk skoru döndürülür (minimizasyon için)


In [35]:
import pyswarms as ps

In [ ]:
# PSO algoritması ile öznitelik seçimi
num_features = X_train.shape[1]
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=num_features, options=options)
best_cost, best_features = optimizer.optimize(objective_function, iters=100)

In [ ]:
# Seçilen en iyi öznitelikleri kullanarak modeli eğitme
model = RandomForestClassifier()
model.fit(X_train.iloc[:, best_features], y_train)

In [ ]:
# Modeli test etme
y_pred = model.predict(X_test.iloc[:, best_features])

In [ ]:
# Doğruluk skorunu hesaplama
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)